In [9]:
import pandas as pd
#remove the lines with n_packets = 0 
df = pd.read_csv('list_of_executed_repositories.csv', delimiter=';', quotechar='"')
df = df[df['n_packets'] != 0]
# Keep rows where the repository column contains 'mongo' or 'mongodb' (case insensitive)
df = df[df['repository'].str.contains('mongo', case=False, na=False)]


df_mongo = df.copy()
df_mongo

,provider,repository,version,compose-file,success,n_packets
712,github,TayPark/mongo-replica-setup,9c82e3c3e167129cf35d56948febc7cdecd84b17,docker-compose.yml,True,50
3187,github,EKI-INDRADI/eki-latihan-mongodb,b363f0eea6053952c7020c8bed2467682b5e4efa,Programmer_Zaman_Now/mongodb/docker-compose.yml,True,7
3188,github,EKI-INDRADI/eki-latihan-mongodb,b363f0eea6053952c7020c8bed2467682b5e4efa,Programmer_Zaman_Now/mongodb-unsecure/docker-c...,True,7
3443,github,Unthrottled/mongo-images,9b18924f9e32c1423b90f080c74a1a0dd86ef07d,docker-compose-build.yml,True,16901
3444,github,Unthrottled/mongo-images,9b18924f9e32c1423b90f080c74a1a0dd86ef07d,docker-compose-deploy.yml,True,7
...,...,...,...,...,...,...
767088,github,sleeepyjack/mongodb,cc7e8253ba275b2565d5f5484bf778aa42539f5f,docker-compose.yml,True,125
767177,github,sonyarianto/docker-compose-mongodb,c9f67e0e27441e2eec4d3734165d345e3fcfcb01,docker-compose.yml,True,5
767403,github,b2wads/mongodb-to-rabbitmq,7a36f162bef993938a7939e41e036277fa3e03ba,docker-compose.yml,True,5
767506,github,korram/demo-nodejs-mongodb-rest,41113e5141e1ada01aac63a3b426aa8f90fea6ca,docker-compose.yml,True,333


Random Stuff from when I was experimenting with the GitHub API (if anyone wanted to debug)

In [7]:
!pip install requests


Defaulting to user installation because normal site-packages is not writeable


In [11]:
import requests
from bs4 import BeautifulSoup
# Function to search for a GitHub repo and check for docker-compose.yml
def find_github_repo_and_docker_compose(repo_name):
    search_url = f"https://api.github.com/search/repositories?q={repo_name}"
    response = requests.get(search_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to search for {repo_name}")
        return None, False
    
    search_results = response.json()
    if search_results['total_count'] == 0:
        print(f"No repositories found for {repo_name}")
        return None, False
    
    # Get the first repository in the search results
    repo = search_results['items'][0]
    repo_full_name = repo['full_name']
    print(f"Found repository: {repo_full_name}")
    
    # Check for the presence of docker-compose.yml
    contents_url = f"https://api.github.com/repos/{repo_full_name}/contents"
    response = requests.get(contents_url)
    
    if response.status_code != 200:
        print(f"Error: Unable to access contents of {repo_full_name}")
        return repo_full_name, False
    
    contents = response.json()
    for file in contents:
        if file['name'] == 'docker-compose.yml':
            print(f"Found docker-compose.yml in {repo_full_name}")
            return repo_full_name, True
    
    print(f"No docker-compose.yml found in {repo_full_name}")
    return repo_full_name, False

# Create lists to store results
repo_full_names = []
docker_compose_exists = []

# Iterate through filtered repos and check for docker-compose.yml
for repo_name in df_mongo['repository']:
    print(f"Searching for {repo_name}...")
    full_name, has_docker_compose = find_github_repo_and_docker_compose(repo_name)
    repo_full_names.append(full_name)
    docker_compose_exists.append(has_docker_compose)

# Add the results to the DataFrame
df_mongo['repo_full_name'] = repo_full_names
df_mongo['docker_compose_exists'] = docker_compose_exists

# Print the final DataFrame
print(df_mongo)

Searching for TayPark/mongo-replica-setup...
Found repository: TayPark/mongo-replica-setup
Found docker-compose.yml in TayPark/mongo-replica-setup
Searching for EKI-INDRADI/eki-latihan-mongodb...
Found repository: EKI-INDRADI/eki-latihan-mongodb
No docker-compose.yml found in EKI-INDRADI/eki-latihan-mongodb
Searching for EKI-INDRADI/eki-latihan-mongodb...
Found repository: EKI-INDRADI/eki-latihan-mongodb
No docker-compose.yml found in EKI-INDRADI/eki-latihan-mongodb
Searching for Unthrottled/mongo-images...
Found repository: Unthrottled/mongo-images
Found docker-compose.yml in Unthrottled/mongo-images
Searching for Unthrottled/mongo-images...
Found repository: Unthrottled/mongo-images
Found docker-compose.yml in Unthrottled/mongo-images
Searching for mv-alex/mongo-passport-local-example...
Found repository: mv-alex/mongo-passport-local-example
Found docker-compose.yml in mv-alex/mongo-passport-local-example
Searching for ioggstream/mongodb-101...
Found repository: ioggstream/mongodb-10

KeyboardInterrupt: 